In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

data = pd.read_csv('10k_Poplar_Tv_Shows.csv')

# data dropped
data = data.drop(columns=['adult'])
data = data.drop(columns=['backdrop_path']) #always drop
data = data.drop(columns=['id']) #always drop
data = data.drop(columns=['original_name']) #always drop
data = data.drop(columns=['overview']) # always drop
data = data.drop(columns=['poster_path']) #always drop
data = data.drop(columns=['first_air_date'])
data = data.drop(columns=['name']) #always drop

# data being used
#data = data.drop(columns=['origin_country'])
#data = data.drop(columns=['original_language'])
#data = data.drop(columns=['popularity'])
#data = data.drop(columns=['genre_ids'])
#data = data.drop(columns=['vote_average'])
#data = data.drop(columns=['vote_count'])

# convert the list-like entry to actual lists or a empty list
data['genre_ids'] = data['genre_ids'].apply(lambda x: eval(x) if isinstance(x, str) else [])

# expand the list of genres 
data_exploded_genres = data.explode('genre_ids')
# encode every column
dummy_genres = pd.get_dummies(data_exploded_genres['genre_ids'], prefix='genre', dtype=int)
dummy_genres = dummy_genres.groupby(dummy_genres.index).sum()
data = data.merge(dummy_genres, left_index=True, right_index=True, how='left')
# all "exploded" columns start with genre as specified by the option prefix='genre'
genre_cols = [col for col in data.columns if col.startswith('genre_')]
data[genre_cols] = data[genre_cols].fillna(0)
data = data.drop(columns=['genre_ids'])


# convert the list-like entry to actual lists or a empty list
data['origin_country'] = data['origin_country'].apply(lambda x: eval(x) if isinstance(x, str) else [])

# expand the list of countries
data_exploded_countries = data.explode('origin_country')
# encode every column
dummy_countries = pd.get_dummies(data_exploded_countries['origin_country'], prefix='country', dtype=int)
dummy_countries = dummy_countries.groupby(dummy_countries.index).sum()
data = data.merge(dummy_countries, left_index=True, right_index=True, how='left')
# all "exploded" columns start with country as specified by the option prefix='country'
country_cols = [col for col in data.columns if col.startswith('country_')]
data[country_cols] = data[country_cols].fillna(0)
data = data.drop(columns=['origin_country'])


data = pd.get_dummies(data, columns=['original_language'], prefix='lang', dtype=int)

data.dropna(inplace=True)


# everything but vote_average
feature_columns = ['popularity', 'vote_count'] + [col for col in data.columns if col.startswith('genre_')] + [col for col in data.columns if col.startswith('country_')] +  [col for col in data.columns if col.startswith('lang_')]

X = data[feature_columns]
y = data['vote_average']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=444)

regressor = RandomForestRegressor(n_estimators=100)
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root mean Squared Error: {rmse:.2f}")

Root mean Squared Error: 1.12
